In [15]:
import BioSimSpace as bss
import glob
import functions as fn
from BioSimSpace import _Exceptions

In [11]:
ligand_path = "../kpc2/inputs/ligands/"
solvated_ligand_files = glob.glob(ligand_path + f"ligand_1_solvated.*")

print(solvated_ligand_files)
test_work_dir = "test_min/"
minimisation_steps = 250

['../kpc2/inputs/ligands/ligand_1_solvated.top', '../kpc2/inputs/ligands/ligand_1_solvated.gro']


In [31]:
solvated_ligand = bss.IO.readMolecules(solvated_ligand_files)

ligand_minimisation_protocol = bss.Protocol.Minimisation(steps=minimisation_steps)
ligand_minimisation_process = bss.Process.Gromacs(solvated_ligand,
                                                  ligand_minimisation_protocol,
                                                  name="min",
                                                  work_dir=test_work_dir)

# fn.edit_mdp_options(test_work_dir, "min", {"emstep": 0.001, "emtol": 1000})
ligand_minimisation_process.setConfig(["emstep = 0.001", "emtol = 2000"])

In [32]:
ligand_minimisation_process.getConfig()

['emstep = 0.001', 'emtol = 2000']

In [33]:
ligand_minimisation_process.start()

# Wait for the process to exit.
ligand_minimisation_process.wait()

# Check for errors.
if ligand_minimisation_process.isError():
    print(ligand_minimisation_process.stdout())
    print(ligand_minimisation_process.stderr())
    raise _Exceptions.ThirdPartyError("The process exited with an error!")

# If it worked, try to get the system. No need to block, since it's already finished.
min_system = ligand_minimisation_process.getSystem()

In [29]:
ligand_minimisation_process.stdout()


Back Off! I just backed up min.gro to ./#min.gro.1#

Steepest Descents converged to Fmax < 1000 in 132 steps
Potential Energy  = -3.1993949e+04
Maximum force     =  8.8761688e+02 on atom 4
Norm of force     =  7.0076917e+01

GROMACS reminds you: "Three Little Fonzies" (Pulp Fiction)



In [22]:
unbound_path = "../kpc2/equilibration/unbound/"
bound_path = "../kpc2/equilibration/bound/"


ligand_1_system_gmx = bss.IO.readMolecules([f"{unbound_path}/ligand_1/npt/npt.gro",
                                            f"{unbound_path}/ligand_1/npt/npt.top"])
ligand_2_system_gmx = bss.IO.readMolecules([f"{unbound_path}/ligand_2/npt/npt.gro",
                                            f"{unbound_path}/ligand_2/npt/npt.top"])



In [7]:
ligand_1_system_gmx.getMolecules()[0].isPerturbable()

False